In [3]:
import pandas as pd

data_dir = 'C:/ZhangLI/Codes/DataSet/个人违贷/official_data/'
train_pub = pd.read_csv(data_dir+'train_public.csv')
train_net = pd.read_csv(data_dir+'train_internet.csv')
test_pub = pd.read_csv(data_dir+'test_public.csv')

import datetime
def preprocessor(dataframe):
    dataframe['issue_date'] = pd.to_datetime(dataframe['issue_date'])
    dataframe['issue_date_y'] = dataframe['issue_date'].dt.year
    dataframe['issue_date_m'] = dataframe['issue_date'].dt.month
    dataframe['issue_date_d'] = dataframe['issue_date'].dt.day
    
    # dataframe['earlies_credit_y'] = dataframe['earlies_credit_mon'].split('-')[1]
    dataframe.drop('earlies_credit_mon', axis = 1, inplace = True)
    # dataframe['earlies_credit_y'] = dataframe['earlies_credit_mon'].split('-')[0].map()
    # set origin date
    # get the diff bewteen now and origin-date
    origin_date = datetime.datetime.strptime('2007-06-01', '%Y-%m-%d')
    dataframe['issue_date_diff'] = dataframe['issue_date'].apply(lambda x: x-origin_date).dt.days
    dataframe.drop('issue_date', axis = 1, inplace = True)
    # 就业类型
    employer_type = dataframe['employer_type'].value_counts().index
    industry = dataframe['industry'].value_counts().index
    # dict()
    emp_type_dict = dict(zip(employer_type, [0,1,2,3,4,5]))
    industry_dict = dict(zip(industry, [i for i in range(15)]))
    # 
    dataframe['work_year'].fillna('10+ years', inplace=True)

    work_year_map = {'10+ years': 10, '2 years': 2, '< 1 year': 0, '3 years': 3, '1 year': 1,
         '5 years': 5, '4 years': 4, '6 years': 6, '8 years': 8, '7 years': 7, '9 years': 9}
    dataframe['work_year']  = dataframe['work_year'].map(work_year_map)

    dataframe['class'] = dataframe['class'].map({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6})

    dataframe['employer_type'] = dataframe['employer_type'].map(emp_type_dict)

    dataframe['industry'] = dataframe['industry'].map(industry_dict)
    return dataframe

# 数据处理
train_net['isDefault'] = train_net['is_default']
common_feature = list(set(train_pub.columns).intersection(set(train_net.columns)))
train_pub_new = train_pub[common_feature]
train_net_new = train_net[common_feature]
train_df = pd.concat([train_pub_new, train_net_new])
# 特征处理
train_df_new = preprocessor(train_df)
# 模型训练
X_train = train_df_new.drop('isDefault', axis=1)
y_train = train_df_new['isDefault']
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(X_train, y_train, test_size=0.2)
import lightgbm as lgb
train_matrix = lgb.Dataset(train_x, label=train_y)
valid_matrix = lgb.Dataset(val_x, label=val_y)
# params = {
#     'boosting_type': 'gbdt',
#             'objective': 'binary',
#             'learning_rate': 0.1,
#             'metric': 'auc',
#             'min_child_weight': 1e-3,
#             'num_leaves': 31,
#             'max_depth': -1,
#             'reg_lambda': 0,
#             'reg_alpha': 0,
#             'feature_fraction': 1,
#             'bagging_fraction': 1,
#             'bagging_freq': 0,
#             'seed': 2020,
#             'nthread': 8,
#             'silent': True,
#             'verbose': -1,
# }

# model = lgb.train(params, train_set=train_matrix, valid_sets=valid_matrix, num_boost_round=2000, verbose_eval=100, early_stopping_rounds=100)

# from sklearn import metrics
# from sklearn.metrics import roc_auc_score

# val_pred_lgb = model.predict(val_x, num_iteration=model.best_iteration)
# fpr, tpr, threshold = metrics.roc_curve(val_y, val_pred_lgb)
# roc_auc = metrics.auc(fpr, tpr)
# import matplotlib.pyplot as plt
# plt.plot(fpr, tpr, 'b', label = 'Val AUC = %0.4f' % roc_auc)
# plt.plot([0, 1], [0,1], 'r--')
# plt.show()
# # 模型预测
# common_feature.remove('isDefault')
# test_pub_new = test_pub[common_feature]
# test_df_new = preprocessor(test_pub_new)
# predict = model.predict(test_df_new, num_iteration=model.best_iteration)
# # 生成结果
# submission = pd.DataFrame({'id':test_pub_new['loan_id'], 'isDefault':predict})
# submission.to_csv('submission.csv', index = None)

In [22]:
# 网格搜索
# Fitting 3 folds for each of 300000 candidates, totalling 900000 fits
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score
import warnings
import warnings
warnings.filterwarnings('ignore')

parameters = {
#     'n_estimators': [500, 1000, 2000, 3000, 5000],
#     'num_leaves': [10,20.30,40,50],
#     'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.15],
    'max_depth': [5, 10],  # default = 31 [5, 10, 15, 20, 25]
#     'min_child_weight': [0, 2, 5, 10, 20],
#     'max_delta_step': [0, 0.2, 0.6, 1, 2],
#     'subsample': [0.6, 0.7, 0.8, 0.85, 0.95],
#     'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9],
#     'reg_alpha': [0, 0.25, 0.5, 0.75, 1],
#     'reg_lambda': [0.2, 0.4, 0.6, 0.8, 1],
#     'scale_pos_weight': [0.2, 0.4, 0.6, 0.8, 1]
}

lgbc = lgb.LGBMClassifier(
                        max_depth=10,
                        learning_rate=0.01,
                        n_estimators=100,
                        silent=True,
                        objective='binary',
                        min_child_weight=1,
                        max_delta_step=0,
                        subsample=0.85,
                        colsample_bytree=0.7,
                        reg_alpha=0,
                        reg_lambda=1,
                        scale_pos_weight=1,
                        seed=1440,
                        num_threads=-1,
                        verbosity=-1
                       )
# Scoring options: [accuracy, f1, precision, recall, roc_auc]
# https://scikit-learn.org/0.22/modules/model_evaluation.html#scoring-parameter
gsearch = GridSearchCV(lgbc, param_grid=parameters, scoring='f1', cv=3, verbose=3, return_train_score=False)
gsearch.fit(train_x, train_y) # traditional train_x / train_y


print("Best score: %0.3f" % gsearch.best_score_)
print("Best parameters set:")
best_parameters = gsearch.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
print('Best estimator:\n', gsearch.best_estimator_)

# print('详细结果:\n', pd.DataFrame.from_dict(gsearch.cv_results_))
# print('最佳分数:\n', gsearch.best_score_)
# print('最佳参数:\n', gsearch.best_params_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV 1/3] END ....................................max_depth=5; total time=   9.5s
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1
[CV 2/3] END ....................................max_depth=5; total time=   8.8s
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1
[CV 3/3] END ....................................max_depth=5; total time=  10.3s
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1
[CV 1/3] END ...................................max_depth=10; total time=  11.5s
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1
[CV 2/3] END ...................................max_depth=10; total time=  10.7s
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1
[CV 3/3] END ..................

In [18]:
# 随机搜索
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV

parameters = {
#     'n_estimators': [500, 1000, 2000, 3000, 5000],
#     'num_leaves': [10,20.30,40,50],
#     'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.15],
    'max_depth': [5, 10],  # default = 31 [5, 10, 15, 20, 25]
#     'min_child_weight': [0, 2, 5, 10, 20],
#     'max_delta_step': [0, 0.2, 0.6, 1, 2],
#     'subsample': [0.6, 0.7, 0.8, 0.85, 0.95],
#     'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9],
#     'reg_alpha': [0, 0.25, 0.5, 0.75, 1],
#     'reg_lambda': [0.2, 0.4, 0.6, 0.8, 1],
#     'scale_pos_weight': [0.2, 0.4, 0.6, 0.8, 1]
}

lgbc = lgb.LGBMClassifier(
                        max_depth=10,
                        learning_rate=0.01,
                        n_estimators=100,
                        silent=True,
                        objective='binary',
                        min_child_weight=1,
                        max_delta_step=0,
                        subsample=0.85,
                        colsample_bytree=0.7,
                        reg_alpha=0,
                        reg_lambda=1,
                        scale_pos_weight=1,
                        seed=1440,
                        num_threads=-1,
                        verbosity=-1
                       )
rand_ser = RandomizedSearchCV(lgbc, param_distributions=parameters, scoring='f1', cv=3, verbose=3)
rand_ser.fit(train_x, train_y)

print("Best score: %0.3f" % rand_ser.best_score_)
print("Best parameters set:")
best_parameters = rand_ser.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
print('Best estimator_:\n', rand_ser.best_estimator_)

# print('详细结果:\n', pd.DataFrame.from_dict(rand_ser.cv_results_))
# print('最佳分数:\n', rand_ser.best_score_)
# print('最佳参数:\n', rand_ser.best_params_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV 1/3] END ....................................max_depth=5; total time=   7.3s
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1
[CV 2/3] END ....................................max_depth=5; total time=   7.4s
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1
[CV 3/3] END ....................................max_depth=5; total time=  10.0s
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=-1 will be ignored. Current value: num_threads=-1
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1
[CV 1/3] END ...................................max_d

In [19]:
# 贝叶斯优化
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import GridSearchCV
import numpy as np

In [28]:
# bayes调参初探
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import GridSearchCV
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import cross_val_score

def lgbc_cv(n_estimators, max_depth,learning_rate,min_child_weight,max_delta_step,subsample,colsample_bytree,reg_alpha,reg_lambda,scale_pos_weight):
    val = cross_val_score(
        lgb.LGBMClassifier(max_depth=int(max_depth),
            learning_rate=min(learning_rate,0.3),
            n_estimators=int(n_estimators),
            #silent=True,
            verbose=4,
            objective='binary',         
            min_child_weight=int(min_child_weight),
            max_delta_step=int(max_delta_step),
            subsample=min(subsample,0.95),
            colsample_bytree=min(colsample_bytree,0.95),
            reg_alpha=min(reg_alpha,1),
            reg_lambda=min(reg_lambda,1),
            scale_pos_weight=min(scale_pos_weight,1),
            seed=1440),
        train_x, train_y, scoring='roc_auc', cv=5
    ).mean()
    return val 

lgbc_bo = BayesianOptimization(
        lgbc_cv,
        {
              'max_depth': (5,100),
              'learning_rate': (0.001,0.3),
              'n_estimators': (500,5000),
              'min_child_weight': (0,20),
              'max_delta_step': (0,2),
              'subsample': (0.1,0.99),
              'colsample_bytree': (0.1,0.99),
              'reg_alpha': (0,1),
              'reg_lambda': (0,1),
              'scale_pos_weight': (0,1)

}
    )
lgbc_bo.maximize(n_iter=5) # 100
lgbc_bo.max

|   iter    |  target   | colsam... | learni... | max_de... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 97057, number of negative: 389343
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.870680
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.271292
[LightGBM] [Debug] init for col-wise cost 0.037471 seconds, init for row-wise cost 0.213219 seconds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.221054 seconds.
You can set 

Exception ignored on calling ctypes callback function: <function _log_callback at 0x0000013FB5F9EEE0>
Traceback (most recent call last):
  File "c:\zhangli\software\installer\python38\lib\site-packages\lightgbm\basic.py", line 76, in _log_callback
    def _log_callback(msg):
KeyboardInterrupt: 


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 16 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 8
Trained a tree with leaves = 31 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves =

KeyboardInterrupt: 

In [30]:
lgbc_bo.max

{}